## Installing Selenium

In [1]:
! pip install selenium

## First attempt

In [2]:
from selenium import webdriver

In [232]:
import glob
import os
import shutil

In [205]:
#downpath = 'C:\\Users\\andre.muchon\\Downloads\\Code\\Scraping Curva DI\\DownDI\\'

In [206]:
#chromeOpt = webdriver.ChromeOptions()
#prefs = {'download.default_directory' : 'downpath'}
#chromeOpt.add_experimental_option("prefs", prefs)
#driver = webdriver.Chrome(chrome_options = chromeOpt)

C:\Users\andre.muchon\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: use options instead of chrome_options
  after removing the cwd from sys.path.


In [226]:
driver = webdriver.Chrome()

In [227]:
driver.get('http://www2.bmf.com.br/pages/portal/bmfbovespa/lumis/lum-taxas-referenciais-bmf-ptBR.asp')

In [228]:
data_area = driver.find_element_by_xpath('//*[@id="Data"]')

In [229]:
data_area.clear()
data_area.send_keys('02/01/2019')

It looks fine. Now lets make sure we can download the file from the website we want

## Getting files

In [220]:
downbutton = driver.find_elements_by_xpath('//*[@id="lumis-taxas-ref-br"]/div/div/div[1]/div[2]/p/a') #get the download button

In [221]:
downbutton[0].click()

In [223]:
import glob

In [ ]:
list_of_files = glob.glob('C:\\Users\\andre.muchon\\Downloads\\')

In [230]:
okbutton = driver.find_elements_by_xpath('//*[@id="divContainerIframeBmf"]/form/div/div/div[1]/div[2]/div/div[2]') #get the ok button

In [231]:
okbutton[0].click() #click

It looks fine. Now lets make sure we can download the file from the website we want

#### Getting files

In [233]:
downbutton = driver.find_elements_by_xpath('//*[@id="lumis-taxas-ref-br"]/div/div/div[1]/div[2]/p/a') #get the download button

In [234]:
downbutton[0].click()

In [237]:
list_of_files = glob.glob('C:\\Users\\andre.muchon\\Downloads\\*')
latest_file = max(list_of_files, key = os.path.getctime)

In [239]:
shutil.copy(latest_file, 'C:\\Users\\andre.muchon\\Downloads\\Code\\Scraping Curva DI\\DownDI\\')

'C:\\Users\\andre.muchon\\Downloads\\Code\\Scraping Curva DI\\DownDI\\PRE20190102 .xls'

### It works!

Let's get a workday list.

In [97]:
import pandas as pd
import os
import datetime
import numpy as np

In [87]:
os.getcwd()

'C:\\Users\\andre.muchon\\Downloads\\Code\\Scraping Curva DI'

In [88]:
path = 'C:\\Users\\andre.muchon\\Downloads\\Code\\Scraping Curva DI'

In [115]:
wd = pd.read_excel('C:\\Users\\andre.muchon\\Downloads\\Code\\Scraping Curva DI\\workdays.xlsx')

In [119]:
wd = wd[ wd.Workdays >= np.datetime64('2018-01-02')] #the first date isn't actually a workday

In [141]:
def createinputdate(e):
    dt = str(e)[:10].split('-')
    dt.reverse()
    inputdate = ''
    for d in dt:
        inputdate = inputdate + d
        inputdate = inputdate + '/'
    return inputdate[:-1]

In [147]:
wd['Data'] = wd.Workdays.map(createinputdate)

In [148]:
wd.head()

,Workdays,Data
1,2018-01-02,02/01/2018
2,2018-01-03,03/01/2018
3,2018-01-04,04/01/2018
4,2018-01-05,05/01/2018
5,2018-01-08,08/01/2018


It looks fine. Now lets make sure we can download the file from the website we want

### Trying to work with beautiful soup

In [240]:
! pip install beautifulsoup4

In [241]:
import requests

In [245]:
from bs4 import BeautifulSoup

In [250]:
import urllib.request

In [242]:
url = 'http://www2.bmf.com.br/pages/portal/bmfbovespa/lumis/lum-taxas-referenciais-bmf-ptBR.asp'

In [251]:
html = urllib.request.urlopen(url).read()

In [252]:
bs = BeautifulSoup(html)

In [264]:
table = bs.find(lambda tag: tag.name == 'table' and tag.has_attr('id') and tag['id'] == 'tb_principal1' )

In [271]:
rows = []
for row in table.findAll("td"):
    rows.append(row)

In [275]:
def parsing(l):
    l_p = []
    for i in range(0,len(l),3):
        l_p.append(l[i:i+3])
    return l_p

In [276]:
rows_p = parsing(rows)

### another try

In [280]:
import requests

In [279]:
url = 'http://www2.bmf.com.br/pages/portal/bmfbovespa/lumis/lum-taxas-referenciais-bmf-ptBR.asp'

In [281]:
w_url = requests.get(url).text

In [284]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(w_url, 'lxml')

In [287]:
my_table = soup.find('table', {'id': 'tb_principal1'})

In [296]:
soup2 = BeautifulSoup(w_url, "html.parser")
infoslist = []
for tr in soup.findAll("table"):
    for td in tr.find_all("td"):
        if not td.attrs.get('style'):
            infoslist.append(td.text)

In [297]:
infoslist_p = parsing(infoslist)

In [303]:
di = pd.DataFrame(infoslist_p)

In [305]:
rename_cols = {0: 'Vertices', 1: 'DIxPRE 252', 2: 'DIxPRE 360'}
di.rename(columns = rename_cols, inplace = True)

In [308]:
di.head()

,Vertices,DIxPRE 252,DIxPRE 360
0,3,"4,90","0,00"
1,7,"4,90","5,00"
2,12,"4,90","4,66"
3,17,"4,90","4,10"
4,21,"4,90","4,66"


It's alive!